In [1]:
import numpy as np
import tensorflow as tf
import random
import sys, os
sys.path.append('/home/sota/EMGcode/EMG_experiment/')
import mypreprocessing
from generator2 import DataGenerator
from models import getNetwork
from custom_layers import MeanOverTime
import sys, os
import json
import scipy
import keras
import tensorflow as tf
from keras import optimizers, initializers, regularizers, constraints
from utils import *
from sklearn import metrics
import matplotlib as plt
from keras.layers import *
from tcn import TCN
from keras.models import Model
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyp
from sklearn.metrics import classification_report

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

os.getcwd()

folder_process_data = '/home/sota/EMGcode/EMG_experiment/DB4_pre/'

os.environ['PYTHONHASHSEED'] = '0'

print('Keras:', keras.__version__)
print('Tensorflow:', tf.__version__)

# 1. Logging
'''
if len(sys.argv) == 4:
    CONFIG_FILE = str(sys.argv[1])
    SUBJECT = int(sys.argv[2])
    TIMESTAMP = int(sys.argv[3])
else:
    print('Expected different number of arguments. {} were given'.format(len(sys.argv) - 1))
    sys.exit()
    '''
CONFIG_FILE='/home/sota/EMGcode/EMG_experiment/config/TCCNet_aot_2 (1).json'

SUBJECT=1
GESTURES=52

with open(CONFIG_FILE) as json_file:
    config_params = json.load(json_file)
outdir='/home/sota/EMGcode/EMG_experiment/DB9_pre/Result'
LOGGING_FILE_PREFIX = config_params['logging']['log_file']
if config_params['logging']['enable']:
    LOGGING_FILE = '/home/sota/EMGcode/EMG_experiment/Result/L_' + LOGGING_FILE_PREFIX + '.log'# 'Musa_NinaPro_Project/ProcessedData/DB1/Result/L_' + LOGGING_FILE_PREFIX + '.log'
    LOGGING_TENSORBOARD_FILE = '/home/sota/EMGcode/EMG_experiment/Result/tblogs/L_' + LOGGING_FILE_PREFIX #'Musa_NinaPro_Project/ProcessedData/DB1/Result/tblogs/L_' + LOGGING_FILE_PREFIX

if config_params['model']['save']:
    MODEL_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/models/O1_' + LOGGING_FILE_PREFIX + '_{}.json'
    MODEL_WEIGHTS_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/models/O2_' + LOGGING_FILE_PREFIX + '_{}.h5'

METRICS_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/metrics/O3_' + LOGGING_FILE_PREFIX + '_{}.mat'

if not os.path.exists(os.path.dirname(METRICS_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(METRICS_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(MODEL_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(MODEL_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise
            
if not os.path.exists(os.path.dirname(LOGGING_TENSORBOARD_FILE)):
    try:
        os.makedirs(os.path.dirname(LOGGING_TENSORBOARD_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

print('Logging file: {}'.format(LOGGING_FILE))
print('Tensorboard file: {}'.format(LOGGING_TENSORBOARD_FILE))
print('Model JSON file: {}'.format(MODEL_SAVE_FILE))
print('Model H5 file: {}'.format(MODEL_WEIGHTS_SAVE_FILE))
print('Metrics file: {}'.format(METRICS_SAVE_FILE))

# 2. Config params generator
PARAMS_TRAIN_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('train_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_TRAIN_GENERATOR[key] = params_gen[key]

PARAMS_VALID_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('valid_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_VALID_GENERATOR[key] = params_gen[key]

2024-11-27 13:25:07.321617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 13:25:07.566760: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-27 13:25:08.289908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-11-27 13:25:08.289973: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

1 Physical GPUs, 1 Logical GPUs
Keras: 2.11.0
Tensorflow: 2.11.0
Logging file: /home/sota/EMGcode/EMG_experiment/Result/L_TCCNet_att.log
Tensorboard file: /home/sota/EMGcode/EMG_experiment/Result/tblogs/L_TCCNet_att
Model JSON file: /home/sota/EMGcode/EMG_experiment/Result/models/O1_TCCNet_att_{}.json
Model H5 file: /home/sota/EMGcode/EMG_experiment/Result/models/O2_TCCNet_att_{}.h5
Metrics file: /home/sota/EMGcode/EMG_experiment/Result/metrics/O3_TCCNet_att_{}.mat


2024-11-27 13:25:09.887571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:10.085271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:10.087601: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:10.090466: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

In [2]:
from keras import backend as K, regularizers, constraints, initializers
from tensorflow.keras.layers import Layer

def scaled_dot_product_attention(q, k, v, mask):
  """アテンションの重みの計算
  q, k, vは最初の次元が一致していること
  k, vは最後から2番めの次元が一致していること
  マスクは型（パディングかルックアヘッドか）によって異なるshapeを持つが、
  加算の際にブロードキャスト可能であること
  引数：
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: (..., seq_len_q, seq_len_k) にブロードキャスト可能な
          shapeを持つ浮動小数点テンソル。既定値はNone

  戻り値：
    出力、アテンションの重み
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # matmul_qkをスケール
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # マスクをスケール済みテンソルに加算
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax は最後の軸(seq_len_k)について
  # 合計が1となるように正規化
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

class ChannelAttention(Layer):
    def __init__(self, ratio = 8):
        super(ChannelAttention, self).__init__()
        self.ratio = ratio

    def build(self, input_shape):
        self.channels = input_shape[-1]

        self.dense = tf.keras.layers.Dense(units=self.channels // self.ratio,
                                           activation='relu',
                                           kernel_initializer='he_normal',
                                           use_bias=True)
        self.dense_output = tf.keras.layers.Dense(units=self.channels,
                                                  kernel_initializer='he_normal',
                                                  use_bias=True)

    def call(self, inputs):
        # inputs = tf.expand_dims(inputs, axis=2)
        gap = tf.reduce_mean(inputs, axis=[0, 1], keepdims=True)

        attention = self.dense(gap)
        attention = self.dense_output(attention)
        attention = tf.keras.activations.sigmoid(attention)

        output = tf.multiply(inputs, attention)

        return output

In [3]:
target_subject = 4

In [4]:
import numpy as np
import tensorflow as tf
import random
import os
os.chdir('/home/sota/EMGcode/EMG_experiment/')
import mypreprocessing
from generator2 import DataGenerator
#from models import getNetwork
from DB2_Mydataset import *
#from model.ninapro_network import *
#from models import TCCNet
from models import getNetwork
import sys
import json
import scipy
import keras
import tensorflow as tf
from keras import optimizers, initializers, regularizers, constraints
from utils import *
from sklearn import metrics
# print(DEFAULT_GENERATOR_PARAMS)
os.getcwd()

import argparse
#import torch
# parser = argparse.ArgumentParser()

# parser.add_argument("-b", "--batch_size", type=int, default=128)  # 16
# parser.add_argument("-lr", "--learning_rate", type=float, default=1e-3)
# parser.add_argument('--cuda', default=True, help='enables cuda')
# parser.add_argument('-j', '--workers', default=8, type=int, metavar='N',
#                     help='number of data loading workers (default: 8)')
# parser.add_argument('--epochs', default=1000, type=int, metavar='N',
#                     help='number of total epochs to run')  # 1000

# parser.add_argument('--patiences', default=500, type=int,
#                     help='number of epochs to tolerate no improvement of val_loss')  # 1000

# '''
# parser.add_argument('--test_subject_id', type=int, default=3,
#                     help='id of test subject, for cross-validation')

# parser.add_argument('--data_cfg', type=int, default=1,
#                     help='0 for 14 class, 1 for 28')

# '''
# parser.add_argument('--dp_rate', type=float, default=0.1,
#                     help='dropout rate')  # 1000

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926
folder_process_data = '/home/sota/EMGcode/EMG_experiment/DB4_pre'

import os
os.environ['PYTHONHASHSEED'] = '0'
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(1234)
# The below is necessary for starting core Python generated random numbers
# in a well-defined state.
random.seed(12345)
# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see:
#    https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res
import tensorflow as tf
from tensorflow import keras

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=0,
    inter_op_parallelism_threads=0
)
session_conf.gpu_options.allow_growth = True
from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see:
#   https://www.tensorflow.org/api_docs/python/tf/set_random_seed
tf.random.set_seed(1234)
#tf.compat.v1.Session()
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

##############################################################################

# 1. Logging
'''
if len(sys.argv) == 4:
    CONFIG_FILE = str(sys.argv[1])
    SUBJECT = int(sys.argv[2])
    TIMESTAMP = int(sys.argv[3])
else:
    print('Expected different number of arguments. {} were given'.format(len(sys.argv) - 1))
    sys.exit()
    '''
CONFIG_FILE='/home/sota/EMGcode/EMG_experiment/config/TCCNet_aot_2 (1).json'

SUBJECT=1
GESTURES=52


with open(CONFIG_FILE) as json_file:
    config_params = json.load(json_file)
outdir='/home/sota/EMGcode/EMG_experiment/DB9_pre/Result'
LOGGING_FILE_PREFIX = config_params['logging']['log_file']
if config_params['logging']['enable']:
    LOGGING_FILE = '/home/sota/EMGcode/EMG_experiment/Result/L_' + LOGGING_FILE_PREFIX + '.log'# 'Musa_NinaPro_Project/ProcessedData/DB1/Result/L_' + LOGGING_FILE_PREFIX + '.log'
    LOGGING_TENSORBOARD_FILE = '/home/sota/EMGcode/EMG_experiment/Result/tblogs/L_' + LOGGING_FILE_PREFIX #'Musa_NinaPro_Project/ProcessedData/DB1/Result/tblogs/L_' + LOGGING_FILE_PREFIX

if config_params['model']['save']:
    MODEL_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/models/O1_' + LOGGING_FILE_PREFIX + '_{}.json'
    MODEL_WEIGHTS_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/models/O2_' + LOGGING_FILE_PREFIX + '_{}.h5'

METRICS_SAVE_FILE = '/home/sota/EMGcode/EMG_experiment/Result/metrics/O3_' + LOGGING_FILE_PREFIX + '_{}.mat'


if not os.path.exists(os.path.dirname(METRICS_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(METRICS_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

if not os.path.exists(os.path.dirname(MODEL_SAVE_FILE)):
    try:
        os.makedirs(os.path.dirname(MODEL_SAVE_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise
            
if not os.path.exists(os.path.dirname(LOGGING_TENSORBOARD_FILE)):
    try:
        os.makedirs(os.path.dirname(LOGGING_TENSORBOARD_FILE))
    except OSError as exc: # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise
            
# 2. Config params generator
PARAMS_TRAIN_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('train_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_TRAIN_GENERATOR[key] = params_gen[key]
# print(PARAMS_TRAIN_GENERATOR)
PARAMS_VALID_GENERATOR = DEFAULT_GENERATOR_PARAMS.copy()
params_gen = config_params['dataset'].get('valid_generator', {}).copy()
for key in params_gen.keys():
    PARAMS_VALID_GENERATOR[key] = params_gen[key]

# 3. Initialization
#INPUT_DIRECTORY = '../dataset/Ninapro-DB1-Proc'
INPUT_DIRECTORY = folder_process_data        #link of process dataset
PARAMS_TRAIN_GENERATOR['preprocess_function_1'] = [mypreprocessing.lpf]
PARAMS_TRAIN_GENERATOR['preprocess_function_1_extra'] = [{'fs': 100}]
PARAMS_TRAIN_GENERATOR['data_type'] = 'rms'
PARAMS_TRAIN_GENERATOR['classes'] = [i for i in range(1, 53)]
PARAMS_TRAIN_GENERATOR['dim'] = [None,12]
PARAMS_TRAIN_GENERATOR['repetitions'] = [1, 3, 4, 6]
PARAMS_TRAIN_GENERATOR['size_factor'] = 10
# PARAMS_TRAIN_GENERATOR['min_max_norm'] = True

PARAMS_VALID_GENERATOR['preprocess_function_1'] = [mypreprocessing.lpf]
PARAMS_VALID_GENERATOR['preprocess_function_1_extra'] = [{'fs': 100}]
PARAMS_VALID_GENERATOR['data_type'] = 'rms'
PARAMS_VALID_GENERATOR['classes'] = [i for i in range(1, 53)]
PARAMS_VALID_GENERATOR['dim'] = [None,12]
PARAMS_VALID_GENERATOR['repetitions'] = [2,5]
PARAMS_VALID_GENERATOR['size_factor'] = 10
# PARAMS_VALID_GENERATOR['min_max_norm'] = True

# PARAMS_TRAIN_GENERATOR['batch_size'] = int(8)
# PARAMS_VALID_GENERATOR['batch_size'] = int(1)
PARAMS_TRAIN_GENERATOR['batch_size'] = int(128/4)
PARAMS_VALID_GENERATOR['batch_size'] = int(128/4)
learning_late = config_params['training']['l_rate']

# PARAMS_TRAIN_GENERATOR['window_size'] = 1000
# PARAMS_VALID_GENERATOR['window_size'] = 1000

# PARAMS_TRAIN_GENERATOR['window_step'] = 500
# PARAMS_VALID_GENERATOR['window_step'] = 500

SUBJECTS = config_params['dataset'].get('subjects', [i for i in range(1, 11)])
if np.min(SUBJECTS) <= 0 or np.max(SUBJECTS) >= 11:
    raise AssertionError('Subject IDs should be between 1 and 27 inclusive for DB1. Were given {}\n'.format(SUBJECTS))

PARAMS_TRAIN_GENERATOR.pop('input_directory', '')
PARAMS_VALID_GENERATOR.pop('input_directory', '')


#MODEL = getNetwork(config_params['model']['name'])

mean_train, mean_test, mean_test_3, mean_test_5 = [], [], [], []
mean_cm = []
mean_train_loss, mean_test_loss = [], []

if config_params['logging']['enable']:
    if os.path.isfile(LOGGING_FILE) == False:
        with open(LOGGING_FILE, 'w') as f:
            f.write(
                'TIMESTAMP: {}\n'
                'KERAS: {}\n'
                'TENSORFLOW: {}\n'
                'DATASET: {}\n'
                'TRAIN_GENERATOR: {}\n'
                'VALID_GENERATOR: {}\n'
                'MODEL: {}\n'
                'MODEL_PARAMS: {}\n'
                'TRAIN_PARAMS: {}\n'.format(
                    keras.__version__, tf.__version__,
                    config_params['dataset']['name'], PARAMS_TRAIN_GENERATOR,
                    PARAMS_VALID_GENERATOR,
                    config_params['model']['name'], config_params['model']['extra'],
                    config_params['training']
                )
            )
            f.write(
                'SUBJECT,TRAIN_SHAPE,TEST_SHAPE,TRAIN_LOSS,TRAIN_ACC,TEST_LOSS,TEST_ACC,TEST_TOP_3_ACC,TEST_TOP_5_ACC\n')

print(f'Subject: {target_subject}')
input_dir = '{}/subject-{:02d}'.format(INPUT_DIRECTORY, target_subject)
# print(input_dir)

train_generator = DataGenerator(input_directory=input_dir, **PARAMS_TRAIN_GENERATOR)
valid_generator = DataGenerator(input_directory=input_dir, **PARAMS_VALID_GENERATOR)

X_train, Y_train, train_reps,L_=train_generator.get_data()
# print(X_train[1].shape)   # X_train_shape=   2176*14221*12
# print(Y_train.shape)   #Y_train_shape= 2176*50
# print(Y_train)
# print(L_.shape)
# print(L_)

y_train=np.where(Y_train==1)[1]
# print(y_train.shape)

X_test, Y_test, test_reps = valid_generator.get_data()     
                             # X test,  Y test
# print(X_test.shape)  #shape   X=100*13083, 12  it is for 2 reptation  Y=100*50   
# print(Y_test.shape)
y_test=np.where(Y_test==1)[1]

# print(y_test.shape)

#reshape of x_train    reading*sample*channel*1
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],1)

# print(X_train.shape[0])
train_dataset = Hand_Dataset(X_train, y_train)   # for 1*8*22*3  label=1
# print(train_dataset[1]['skeleton'].shape)
    
# print('Call Mydataset for testing :')
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1],X_test.shape[2],1)
test_dataset =  Hand_Dataset(X_test, y_test)
# print(test_dataset[1]['skeleton'].shape)

# args = parser.parse_args()
# args, unknown = parser.parse_known_args()

# train_loader = torch.utils.data.DataLoader(
#     train_dataset,
#     batch_size=args.batch_size, shuffle=True,
#     num_workers=args.workers, pin_memory=False)

# val_loader = torch.utils.data.DataLoader(
#     test_dataset,
#     batch_size=args.batch_size, shuffle=False,
#     num_workers=args.workers, pin_memory=False)

class_num=train_generator.n_classes
# print(class_num)

timelen=X_train.shape[1]

# print(timelen)

# print('Train generator:')
# print(train_generator)
# print('Test generator:')
# print(valid_generator)
# print(train_generator.n_classes)
# print(train_generator.repetitions)

print('ok')

2024-11-27 13:25:13.229409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:13.230977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:13.232512: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:13.233941: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-27 13:25:13.235397: I tensorflow/compiler/xla/stream_executo

Subject: 4


IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
from keras.layers import *
from tcn import TCN
from keras.models import Model

tf.random.set_seed(1234)

def se_block(in_block, ch, ratio=2):
    z = GlobalAveragePooling1D()(in_block) 
    x = Dense(ch // ratio, activation='relu')(z)
    x = Dense(ch, activation='sigmoid')(x)
    return multiply([in_block, x])

input_shape = (None, 12)

input = Input(shape=input_shape)

branch1 = TCN(nb_filters=32, nb_stacks=1, kernel_size=3, return_sequences=True)(input)
branch1 = Bidirectional(LSTM(32, return_sequences=True))(branch1)

branch2 = Conv1D(2, 3)(input)
branch2 = SeparableConv1D(filters=2, kernel_size=3, padding='same', activation='relu')(branch2)
branch2 = se_block(branch2, ch=2)

branch3 = Bidirectional(TCN(nb_filters=32, nb_stacks=1, kernel_size=3, return_sequences=True))(input)

# Flatten the sequences before concatenating
branch1_flat = GlobalAveragePooling1D()(branch1)
branch2_flat = GlobalAveragePooling1D()(branch2)
branch3_flat = GlobalAveragePooling1D()(branch3)
# input_flat = GlobalAveragePooling1D()(input)

x = concatenate([branch1_flat, branch2_flat, branch3_flat])
# x = concatenate([branch1_flat, branch2_flat, branch3_flat, input_flat])
x = Dropout(0.2)(x)
x = ChannelAttention()(x)
output = Dense(52, trainable=True, activation='softmax')(x)

model = Model(inputs=input, outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 12)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, None, 2)      74          ['input_1[0][0]']                
                                                                                                  
 separable_conv1d (SeparableCon  (None, None, 2)     12          ['conv1d[0][0]']                 
 v1D)                                                                                             
                                                                                                  
 global_average_pooling1d (Glob  (None, 2)           0           ['separable_conv1d[0][0]']   

In [6]:
tf.random.set_seed(1234)
print(target_subject)
from tensorflow.keras.callbacks import Callback

class SavePredictionsCallback(Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test
        self.predictions_per_epoch = []
        self.true_labels_per_epoch = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.X_test)
        self.predictions_per_epoch.append(y_pred)
        self.true_labels_per_epoch.append(self.y_test)
        print(f"Epoch {epoch+1}: saved predictions and true labels")
        
if config_params['training']['optimizer'] == 'adam':
    optimizer = optimizers.Adam(learning_rate=learning_late, epsilon=0.001)
    
elif config_params['training']['optimizer'] == 'sgd':
    optimizer = optimizers.SGD(learning_rate=learning_late, momentum=0.9)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy', top_3_accuracy, top_5_accuracy])

train_callbacks = []

lrScheduler = MyLRScheduler(**config_params['training']['l_rate_schedule'])
train_callbacks.append(lrScheduler)
#print(train_generator)

save_predictions_callback = SavePredictionsCallback(X_test, Y_test)

history = model.fit(train_generator, epochs=128, 
                              validation_data=(X_test,Y_test), callbacks=[save_predictions_callback], verbose=2)

Y_pred = model.predict(X_test)

y_pred = np.argmax(Y_pred, axis=1)
y_test = np.argmax(Y_test, axis=1)
    
#print(history.history)

#acc_list.append(max(history.history['accuracy']))

if config_params['model']['save']:
    # serialize model to JSON
    model_json = model.to_json()
    with open(MODEL_SAVE_FILE.format(SUBJECT), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(MODEL_WEIGHTS_SAVE_FILE.format(SUBJECT))
    print("Saved model to disk")


# Confusion Matrix
# C_{i, j} is equal to the number of observations known to be in group i but predicted to be in group j.
cnf_matrix_frame = metrics.confusion_matrix(y_test, y_pred)
if np.array(mean_cm).shape != cnf_matrix_frame.shape:
    mean_cm = cnf_matrix_frame
else:
    mean_cm += cnf_matrix_frame

#dict_keys(['loss', 'accuracy', 'top_3_accuracy', 'top_5_accuracy', 'val_loss', 'val_accuracy', 'val_top_3_accuracy', 'val_top_5_accuracy'])

mean_train.append(history.history['accuracy'][-1])
mean_test.append(history.history['val_accuracy'][-1])
mean_train_loss.append(history.history['loss'][-1])
mean_test_loss.append(history.history['val_loss'][-1])
mean_test_3.append(history.history['val_top_3_accuracy'][-1])
mean_test_5.append(history.history['val_top_5_accuracy'][-1])

if config_params['logging']['enable']:
    with open(LOGGING_FILE, 'a') as f:
        f.write('{},{},{},{},{},{},{},{},{}\n'.format(SUBJECT, train_generator.__len__() * PARAMS_TRAIN_GENERATOR['batch_size'], valid_generator.__len__(),
            mean_train_loss[-1], mean_train[-1], mean_test_loss[-1], mean_test[-1], mean_test_3[-1], mean_test_5[-1]))

metrics_dict = {
    'mean_cm': mean_cm,
    'mean_test': mean_test,
    'mean_test_3': mean_test_3,
    'mean_test_5': mean_test_5,
    'mean_train': mean_train,
    'mean_train_loss': mean_train_loss,
    'mean_test_loss': mean_test_loss
}
scipy.io.savemat(METRICS_SAVE_FILE.format(SUBJECT), metrics_dict)

9
Epoch 1/128


2024-11-19 16:09:13.632266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8600
2024-11-19 16:09:13.681254: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-11-19 16:09:14.034082: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


3/3 [==============================] - 1s 177ms/step
Epoch 1: saved predictions and true labels
71/71 - 33s - loss: 5.5215 - accuracy: 0.0268 - top_3_accuracy: 0.0726 - top_5_accuracy: 0.1276 - val_loss: 3.8677 - val_accuracy: 0.0521 - val_top_3_accuracy: 0.1042 - val_top_5_accuracy: 0.1979 - 33s/epoch - 466ms/step
Epoch 2/128
3/3 [==============================] - 1s 178ms/step
Epoch 2: saved predictions and true labels
71/71 - 27s - loss: 3.7566 - accuracy: 0.0511 - top_3_accuracy: 0.1501 - top_5_accuracy: 0.2262 - val_loss: 3.5002 - val_accuracy: 0.1146 - val_top_3_accuracy: 0.2396 - val_top_5_accuracy: 0.2917 - 27s/epoch - 387ms/step
Epoch 3/128
3/3 [==============================] - 1s 175ms/step
Epoch 3: saved predictions and true labels
71/71 - 27s - loss: 3.2101 - accuracy: 0.1351 - top_3_accuracy: 0.3116 - top_5_accuracy: 0.4199 - val_loss: 2.6052 - val_accuracy: 0.2396 - val_top_3_accuracy: 0.5312 - val_top_5_accuracy: 0.6979 - 27s/epoch - 387ms/step
Epoch 4/128
3/3 [========

KeyboardInterrupt: 

In [ ]:
print("-----subject-------")
print(target_subject)
print("----accuracy-----")
print(max(history.history['accuracy']))
print("---val accuracy---")
print(max(history.history['val_accuracy']))

predictions_per_epoch = save_predictions_callback.predictions_per_epoch
true_labels_per_epoch = save_predictions_callback.true_labels_per_epoch

for epoch, (y_pred, y_true) in enumerate(zip(predictions_per_epoch, true_labels_per_epoch)):
    y_pred = np.argmax(y_pred, axis=1)
    if (round(metrics.accuracy_score(y_test, y_pred),4) == round(max(history.history['val_accuracy']),4)):
        print(f"Epoch {epoch+1}")
        # print("Predictions:", y_pred)
        # print("True Labels:", y_true)

        report = classification_report(y_test, y_pred, digits=4)
        print(report)

target_subject += 1

In [ ]:
directory = '/home/sota/EMGcode/temp_result'
file_name = f'DB4_report{target_subject-1}.txt'
file_path = os.path.join(directory, file_name)

with open(file_path, 'w') as file:
    file.write(report)

print(f"Classification report saved to {file_path}")

In [9]:
print(PARAMS_TRAIN_GENERATOR)
print('\n---------------------------------------------------------------------------\n')
print(PARAMS_VALID_GENERATOR)
print('\n---------------------------------------------------------------------------\n')
print(train_generator)
print('---------------------------------------------------------------------------\n')
print(valid_generator)

{'repetitions': [1, 3, 4, 6], 'batch_size': 32, 'sample_weight': True, 'dim': [None, 12], 'classes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], 'shuffle': True, 'noise_snr_db': 25, 'scale_sigma': 0.0, 'window_size': 0, 'window_step': 0, 'rotation': 0, 'rotation_mask': None, 'time_warping': 0.2, 'mag_warping': 0.2, 'permutation': 0, 'data_type': 'rms', 'preprocess_function_1': [<function lpf at 0x7898aebc40d0>], 'preprocess_function_2': None, 'preprocess_function_1_extra': [{'fs': 100}], 'preprocess_function_2_extra': None, 'size_factor': 10, 'pad_len': None, 'pad_value': -10.0, 'min_max_norm': True, 'update_after_epoch': False, 'label_proc': None, 'label_proc_extra': None}

---------------------------------------------------------------------------

{'repetitions': [2, 5], 'batch_size': 32, 'sample_weight': False, 'dim': [None, 12

In [7]:
print(PARAMS_TRAIN_GENERATOR)
print('---------------------------------------------------------------------------\n')
print(PARAMS_VALID_GENERATOR)
print('---------------------------------------------------------------------------\n')
print(train_generator)
print('---------------------------------------------------------------------------\n')
print(valid_generator)

{'repetitions': [1, 3, 4, 6], 'batch_size': 32, 'sample_weight': True, 'dim': [None, 12], 'classes': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52], 'shuffle': True, 'noise_snr_db': 25, 'scale_sigma': 0.0, 'window_size': 0, 'window_step': 0, 'rotation': 0, 'rotation_mask': None, 'time_warping': 0.2, 'mag_warping': 0.2, 'permutation': 0, 'data_type': 'rms', 'preprocess_function_1': [<function lpf at 0x728bb9fbc3a0>], 'preprocess_function_2': None, 'preprocess_function_1_extra': [{'fs': 100}], 'preprocess_function_2_extra': None, 'size_factor': 10, 'pad_len': None, 'pad_value': -10.0, 'min_max_norm': True, 'update_after_epoch': False, 'label_proc': None, 'label_proc_extra': None}
---------------------------------------------------------------------------

{'repetitions': [2, 5], 'batch_size': 32, 'sample_weight': False, 'dim': [None, 12]